In [3]:
from mdutils.mdutils import MdUtils
import tqdm
import math
import requests
import pandas as pd
import time
from wikidataintegrator import wdi_core, wdi_login
from rdflib import Graph, Literal, RDF, URIRef, Namespace, BNode
from rdflib.namespace import DCTERMS, FOAF, XSD, DC, RDFS
import shutil
import os

# Load occurence data

In [4]:
df = pd.read_csv('occurrence.tsv', sep='\t', on_bad_lines = 'warn')
df

Skipping line 293712: expected 212 fields, saw 230

Skipping line 377950: expected 212 fields, saw 295
Skipping line 380020: expected 212 fields, saw 213

Skipping line 396956: expected 212 fields, saw 258

Skipping line 470578: expected 212 fields, saw 249

/var/folders/85/6rr6j2113qx4d9synl4rnc9r0000gn/T/ipykernel_2302/3560789328.py:1: DtypeWarning: Columns (1,2,3,5,7,8,9,10,11,12,13,14,15,16,18,19,20,22,23,24,25,28,29,30,31,32,33,36,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,57,59,63,64,65,67,68,69,70,71,72,73,74,75,76,77,79,80,81,82,83,84,86,89,90,91,96,97,98,99,101,102,103,104,105,106,109,111,115,117,118,119,124,125,126,127,128,129,130,131,132,133,134,135,137,138,139,140,141,143,144,145,146,147,149,155,158,159,161,164,165,166,168,169,179,180,181,183,184,185,186,189,190,191,195,199,201,202,203,204,205,206,207,208,209) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('occurrence.tsv', sep='\t', on_bad_lines = 'warn')


,gbifID,accessRights,bibliographicCitation,language,license,modified,publisher,references,rightsHolder,type,...,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name,iucnRedListCategory,eventType
0,2516254663,NaN,NaN,NaN,CC0_1_0,NaN,Naturalis Biodiversity Center,NaN,Naturalis Biodiversity Center,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2513367778,NaN,NaN,NaN,CC0_1_0,NaN,Naturalis Biodiversity Center,NaN,Naturalis Biodiversity Center,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LC,NaN
2,2513369818,NaN,NaN,NaN,CC0_1_0,NaN,Naturalis Biodiversity Center,NaN,Naturalis Biodiversity Center,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NE,NaN
3,2513369962,NaN,NaN,NaN,CC0_1_0,NaN,Naturalis Biodiversity Center,NaN,Naturalis Biodiversity Center,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NE,NaN
4,2513378970,NaN,NaN,NaN,CC0_1_0,NaN,Naturalis Biodiversity Center,NaN,Naturalis Biodiversity Center,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LC,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558530,1803055054,NaN,NaN,pt,CC_BY_NC_4_0,2023-04-06T19:41:46Z,"Instituto de Botânica, São Paulo",NaN,SP-Fungi - Maria Eneyda Pacheco Kauffmann Fidalgo,Collection,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
558531,1803055080,NaN,NaN,pt,CC_BY_NC_4_0,2023-04-06T19:41:21Z,"Instituto de Botânica, São Paulo",NaN,SP-Fungi - Maria Eneyda Pacheco Kauffmann Fidalgo,Collection,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NE,NaN
558532,1803055075,NaN,NaN,pt,CC_BY_NC_4_0,2023-04-06T19:41:51Z,"Instituto de Botânica, São Paulo",NaN,SP-Fungi - Maria Eneyda Pacheco Kauffmann Fidalgo,Collection,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NE,NaN
558533,1803054806,NaN,NaN,pt,CC_BY_NC_4_0,2023-04-06T19:41:47Z,"Instituto de Botânica, São Paulo",NaN,SP-Fungi - Maria Eneyda Pacheco Kauffmann Fidalgo,Collection,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NE,NaN


# Fetch institute QID's for institutes

In [5]:
user_agent = 'TaxonQueries/1.0 (andra@micelio.be) SPARQL queries'  # Replace with your actual User-Agent string

result_df = pd.DataFrame(columns=['Institute', 'Institute_QID'])
batchSize = 5
retry_wait_time = 1  # Initial wait time in seconds
institutes = set(df["publisher"])
instituteQID = dict()
i = 0
notFound = {}
for institute in institutes:
    en = wdi_core.WDItemEngine.get_wd_search_results(institute, language='en')
    if len(en) > 0:
        instituteQID[institute] = en[0]
        continue
    nl = wdi_core.WDItemEngine.get_wd_search_results(institute, language='nl')
    if len(nl) > 0:
        instituteQID[institute] = nl[0]
        continue
    de = wdi_core.WDItemEngine.get_wd_search_results(institute, language='de')
    if len(de) > 0:
        instituteQID[institute] = de[0]
        continue
    fr = wdi_core.WDItemEngine.get_wd_search_results(institute, language='fr')
    if len(fr) > 0:
        instituteQID[institute] = fr[0]
        continue
    es = wdi_core.WDItemEngine.get_wd_search_results(institute, language='es')
    if len(es) > 0:
        instituteQID[institute] = es[0]
        continue
    pt = wdi_core.WDItemEngine.get_wd_search_results(institute, language='pt')
    if len(pt) > 0:
        instituteQID[institute] = pt[0]
        continue
    else:
        i+=1
        print(str(i)+f" Not found institute={institute}")
        notFound[institute] = requests.get("https://api.gbif.org/v1/organization?q="+institute).json()

1 Not found institute=CBGP (UMR INRA, Cirad, IRD, Montpellier SupAgro)
2 Not found institute=Walter Reed Biosystematics Unit, Smithsonian Institution
3 Not found institute=University of Tartu, Natural History Museum and Botanical Garden


In [6]:
df2 = df[df["taxonRank"] == "SPECIES"]
df3 = df2[["species", "taxonKey", "taxonRank"]].drop_duplicates()
species_counts = df3["species"].value_counts()
species_counts

grouped = df3.groupby('species').apply(lambda x: list(x.taxonKey))
for species, taxon_keys in grouped.items():
    if len(taxon_keys) > 1:
        taxon_keys = list(map(int, taxon_keys))
        print(f"Species: {species}, Taxon Keys: {taxon_keys}")

Species: Abelmoschus esculentus, Taxon Keys: [3152707, 3152708]
Species: Abolboda killipii, Taxon Keys: [2692545, 2692543]
Species: Abrodictyum rigidum, Taxon Keys: [3606920, 8069499]
Species: Acalypha diversifolia, Taxon Keys: [3056298, 9321483]
Species: Acestrorhynchus microlepis, Taxon Keys: [2355569, 11027454, 2355573]
Species: Achelous ordwayi, Taxon Keys: [5863627, 5178581]
Species: Achelous spinicarpus, Taxon Keys: [5178579, 5974595]
Species: Achelous tumidulus, Taxon Keys: [5863625, 2225638]
Species: Achlyodes thraso, Taxon Keys: [4301920, 1945452, 1945477]
Species: Acianthera ciliata, Taxon Keys: [7816126, 7548915, 5951596, 2786740]
Species: Acianthera fockei, Taxon Keys: [2840105, 2840103]
Species: Aciotis acuminifolia, Taxon Keys: [3862481, 3862215]
Species: Aciotis annua, Taxon Keys: [3862177, 3862389]
Species: Aciotis indecora, Taxon Keys: [3862080, 3869313, 3862036]
Species: Aciotis ornata, Taxon Keys: [3861937, 3862111, 3869188]
Species: Aciotis purpurascens, Taxon Keys:

In [7]:
taxonname =  df2[["species", "taxonKey", "taxonRank", "taxonRank", "genus", "genericName","taxonomicStatus", "acceptedNameUsageID", "scientificName", "specificEpithet"]].drop_duplicates()
taxonnames = []
for index, row in taxonname.iterrows():
    if isinstance(row["specificEpithet"], str):
        taxonnames.append("\""+row["genericName"] + " " + row["specificEpithet"]+"\"")
    else:
        continue
    taxonnames.append("\""+row["genericName"] + " " + row["specificEpithet"]+"\"")
taxonnames = list(set(taxonnames))

# resolve taxonnames and kingdom

In [ ]:
url = 'https://query.wikidata.org/sparql'
user_agent = 'TaxonQueries/1.0 ( andra@micelio.be) SPARQL queries'  # Replace with your actual User-Agent string

result_df = pd.DataFrame(columns=['taxonId', 'taxonname', 'taxon', 'kingdomname', 'kingdom' ])
batchSize = 10
retry_wait_time = 1  # Initial wait time in seconds

for i in tqdm.tqdm(range(math.ceil(len(taxonnames)/batchSize)), desc="Processing batches"):
    batch = taxonnames[i*batchSize:(i+1)*batchSize]
    query = """
            SELECT ?taxon ?taxonId ?taxonname ?kingdomname ?kingdom ?article
                WHERE {
                    VALUES ?taxonname {""" + " ".join(batch) + """}
                    ?taxon wdt:P225 ?taxonname .
                    ?taxon wdt:P171* ?kingdom .
                    OPTIONAL {?article schema:about ?taxon ;
                               schema:isPartOf <https://en.wikipedia.org/> .}
                    ?kingdom wdt:P105 wd:Q36732 .
                    ?kingdom rdfs:label ?kingdomname filter (lang(?kingdomname) = "en") .
                    OPTIONAL {?taxon wdt:P846 ?taxonId .}
            }
            """
    # print(query)

    success = False
    while not success:
        try:
            headers = {'User-Agent': user_agent}
            r = requests.get(url, params={'format': 'json', 'query': query}, headers=headers)
            r.raise_for_status()  # Raise an HTTPError for bad responses
            data = r.json()
            temp_df = pd.DataFrame([(item.get("taxonId", {}).get("value", ""),
                                     item.get("taxonname", {}).get("value", ""),
                                     item.get("taxon", {}).get("value", ""),
                                     item.get("kingdomname", {}).get("value", ""),
                                     item.get("kingdom", {}).get("value", ""),
                                     item.get("article", {}).get("value", "")
                                    )
                                    for item in data["results"]["bindings"]],
                                    columns=['taxonId', 'taxonname', 'taxon', 'kingdomname', 'kingdom', 'article' ])

            # Concatenate the temporary DataFrame with the main result DataFrame
            result_df = pd.concat([result_df, temp_df], ignore_index=True)
            success = True  # Set to True to break out of the while loop on success

        except requests.exceptions.HTTPError as err:
            print(f"Error: {err}")
            print(f"Retrying in {retry_wait_time} seconds...")
            time.sleep(retry_wait_time)
            retry_wait_time *= 2  # Double the wait time
            if retry_wait_time > 60:  # Limit the maximum wait time to 60 seconds
                retry_wait_time = 60
        else:
            retry_wait_time = 1  # Reset to 1 second after a successful query

# Now, result_df contains all the data
result_df


Processing batches:   0%|          | 3/1970 [00:02<31:43,  1.03it/s]

# Merge taxonnames and kingdom

In [ ]:
result_df[['genericName', 'specificEpithet']] = result_df['taxonname'].str.split(expand=True, n=1)

# Print the updated DataFrame
wdtaxqid = dict()
for index, row in result_df.iterrows():
    if row["taxonId"] != "":
        wdtaxqid[row["taxonId"]] = row["taxon"]
result_df

In [26]:
merged_df = pd.merge(df, result_df, on=['genericName', 'specificEpithet'], how='inner')
merged_df

,gbifID,accessRights,bibliographicCitation,language,license,modified,publisher,references,rightsHolder,type,...,level3Name,iucnRedListCategory,eventType,taxonId,taxonname,taxon,kingdomname,kingdom_y,article,temp_join
0,2516254663,NaN,NaN,NaN,CC0_1_0,NaN,Naturalis Biodiversity Center,NaN,Naturalis Biodiversity Center,NaN,...,NaN,NaN,NaN,5421109,Sauvagesia erecta,http://www.wikidata.org/entity/Q15381185,plant,http://www.wikidata.org/entity/Q756,,Sauvagesia erecta
1,2517199903,NaN,NaN,NaN,CC0_1_0,NaN,Naturalis Biodiversity Center,NaN,Naturalis Biodiversity Center,NaN,...,NaN,LC,NaN,5421109,Sauvagesia erecta,http://www.wikidata.org/entity/Q15381185,plant,http://www.wikidata.org/entity/Q756,,Sauvagesia erecta
2,2515358577,NaN,NaN,NaN,CC0_1_0,NaN,Naturalis Biodiversity Center,NaN,Naturalis Biodiversity Center,NaN,...,NaN,LC,NaN,5421109,Sauvagesia erecta,http://www.wikidata.org/entity/Q15381185,plant,http://www.wikidata.org/entity/Q756,,Sauvagesia erecta
3,2515553384,NaN,NaN,NaN,CC0_1_0,NaN,Naturalis Biodiversity Center,NaN,Naturalis Biodiversity Center,NaN,...,NaN,LC,NaN,5421109,Sauvagesia erecta,http://www.wikidata.org/entity/Q15381185,plant,http://www.wikidata.org/entity/Q756,,Sauvagesia erecta
4,1455978879,NaN,NaN,NaN,CC0_1_0,2019-08-16T14:20:00Z,"National Museum of Natural History, Smithsonia...",NaN,NaN,NaN,...,NaN,LC,NaN,5421109,Sauvagesia erecta,http://www.wikidata.org/entity/Q15381185,plant,http://www.wikidata.org/entity/Q756,,Sauvagesia erecta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468070,1803055072,NaN,NaN,pt,CC_BY_NC_4_0,2023-04-06T19:41:21Z,"Instituto de Botânica, São Paulo",NaN,SP-Fungi - Maria Eneyda Pacheco Kauffmann Fidalgo,Collection,...,NaN,NE,NaN,2544558,Rigidoporus liebmannii,http://www.wikidata.org/entity/Q108327439,fungus,http://www.wikidata.org/entity/Q764,,Rigidoporus liebmannii
468071,1803055080,NaN,NaN,pt,CC_BY_NC_4_0,2023-04-06T19:41:21Z,"Instituto de Botânica, São Paulo",NaN,SP-Fungi - Maria Eneyda Pacheco Kauffmann Fidalgo,Collection,...,NaN,NE,NaN,2544558,Rigidoporus liebmannii,http://www.wikidata.org/entity/Q108327439,fungus,http://www.wikidata.org/entity/Q764,,Rigidoporus liebmannii
468072,1803055051,NaN,NaN,pt,CC_BY_NC_4_0,2023-04-06T19:41:48Z,"Instituto de Botânica, São Paulo",NaN,SP-Fungi - Maria Eneyda Pacheco Kauffmann Fidalgo,Collection,...,NaN,NE,NaN,5237620,Polyporus aculeifera,http://www.wikidata.org/entity/Q108273395,fungus,http://www.wikidata.org/entity/Q764,,Polyporus aculeifera
468073,1803055032,NaN,NaN,pt,CC_BY_NC_4_0,2023-04-06T19:41:12Z,"Instituto de Botânica, São Paulo",NaN,SP-Fungi - Maria Eneyda Pacheco Kauffmann Fidalgo,Collection,...,NaN,NE,NaN,2520540,Phellinus scruposus,http://www.wikidata.org/entity/Q105060807,fungus,http://www.wikidata.org/entity/Q764,,Phellinus scruposus


In [27]:
merged_df[['taxonKey', 'taxonId', 'taxonname', 'taxon', 'genericName', 'specificEpithet', 'kingdom_y', 'kingdomname', 'article']].drop_duplicates()

,taxonKey,taxonId,taxonname,taxon,genericName,specificEpithet,kingdom_y,kingdomname,article
0,5553003.0,5421109,Sauvagesia erecta,http://www.wikidata.org/entity/Q15381185,Sauvagesia,erecta,http://www.wikidata.org/entity/Q756,plant,
1,5421109.0,5421109,Sauvagesia erecta,http://www.wikidata.org/entity/Q15381185,Sauvagesia,erecta,http://www.wikidata.org/entity/Q756,plant,
28,7326816.0,5421109,Sauvagesia erecta,http://www.wikidata.org/entity/Q15381185,Sauvagesia,erecta,http://www.wikidata.org/entity/Q756,plant,
122,3621214.0,3621214,Parahancornia fasciculata,http://www.wikidata.org/entity/Q6060970,Parahancornia,fasciculata,http://www.wikidata.org/entity/Q756,plant,https://en.wikipedia.org/wiki/Parahancornia_fa...
160,7533920.0,5580272,Salacia miqueliana,http://www.wikidata.org/entity/Q15345225,Salacia,miqueliana,http://www.wikidata.org/entity/Q756,plant,
...,...,...,...,...,...,...,...,...,...
468068,2546521.0,2546521,Trametes caperata,http://www.wikidata.org/entity/Q108438435,Trametes,caperata,http://www.wikidata.org/entity/Q764,fungus,
468069,2544558.0,2544558,Rigidoporus liebmannii,http://www.wikidata.org/entity/Q108327439,Rigidoporus,liebmannii,http://www.wikidata.org/entity/Q764,fungus,
468072,5237620.0,5237620,Polyporus aculeifera,http://www.wikidata.org/entity/Q108273395,Polyporus,aculeifera,http://www.wikidata.org/entity/Q764,fungus,
468073,2520540.0,2520540,Phellinus scruposus,http://www.wikidata.org/entity/Q105060807,Phellinus,scruposus,http://www.wikidata.org/entity/Q764,fungus,


In [31]:
taxoninformation = dict()

In [14]:
def get_wikipedia_snippet(title):
    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "format": "json",
        "titles": title,
        "prop": "extracts",
        "exintro": True,
        "explaintext": True
    }
    response = requests.get(url, params=params)
    data = response.json()
    page = next(iter(data['query']['pages'].values()))
    snippet = page.get("extract", "")
    return snippet



In [32]:
query = """
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    SELECT DISTINCT ?taxon  WHERE {
           ?observation wdt:P225 ?taxon .
           ?taxon rdfs:label ?taxonLabel .
            }"""
# results = gbifRDF.query(query)
for taxonname in tqdm.tqdm(taxonname_list):
    if taxonname not in taxoninformation.keys():
        taxoninformation[taxonname] = dict()
        taxoninformation[taxonname]["wikiintroduction"] = get_wikipedia_snippet(taxonname.replace(" ", "_"))


100%|██████████| 5332/5332 [28:34<00:00,  3.11it/s]


In [36]:
taxon_info_df = pd.DataFrame.from_dict(taxoninformation, orient='index').reset_index()
taxon_info_df.rename(columns={'index': 'taxonname'}, inplace=True)

# Merge the dictionary DataFrame with your existing DataFrame
merged_df2 = merged_df.merge(taxon_info_df, on='taxonname', how='left')
merged_df2

,gbifID,accessRights,bibliographicCitation,language,license,modified,publisher,references,rightsHolder,type,...,iucnRedListCategory,eventType,taxonId,taxonname,taxon,kingdomname,kingdom_y,article,temp_join,wikiintroduction
0,2516254663,NaN,NaN,NaN,CC0_1_0,NaN,Naturalis Biodiversity Center,NaN,Naturalis Biodiversity Center,NaN,...,NaN,NaN,5421109,Sauvagesia erecta,http://www.wikidata.org/entity/Q15381185,plant,http://www.wikidata.org/entity/Q756,,Sauvagesia erecta,NaN
1,2517199903,NaN,NaN,NaN,CC0_1_0,NaN,Naturalis Biodiversity Center,NaN,Naturalis Biodiversity Center,NaN,...,LC,NaN,5421109,Sauvagesia erecta,http://www.wikidata.org/entity/Q15381185,plant,http://www.wikidata.org/entity/Q756,,Sauvagesia erecta,NaN
2,2515358577,NaN,NaN,NaN,CC0_1_0,NaN,Naturalis Biodiversity Center,NaN,Naturalis Biodiversity Center,NaN,...,LC,NaN,5421109,Sauvagesia erecta,http://www.wikidata.org/entity/Q15381185,plant,http://www.wikidata.org/entity/Q756,,Sauvagesia erecta,NaN
3,2515553384,NaN,NaN,NaN,CC0_1_0,NaN,Naturalis Biodiversity Center,NaN,Naturalis Biodiversity Center,NaN,...,LC,NaN,5421109,Sauvagesia erecta,http://www.wikidata.org/entity/Q15381185,plant,http://www.wikidata.org/entity/Q756,,Sauvagesia erecta,NaN
4,1455978879,NaN,NaN,NaN,CC0_1_0,2019-08-16T14:20:00Z,"National Museum of Natural History, Smithsonia...",NaN,NaN,NaN,...,LC,NaN,5421109,Sauvagesia erecta,http://www.wikidata.org/entity/Q15381185,plant,http://www.wikidata.org/entity/Q756,,Sauvagesia erecta,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468070,1803055072,NaN,NaN,pt,CC_BY_NC_4_0,2023-04-06T19:41:21Z,"Instituto de Botânica, São Paulo",NaN,SP-Fungi - Maria Eneyda Pacheco Kauffmann Fidalgo,Collection,...,NE,NaN,2544558,Rigidoporus liebmannii,http://www.wikidata.org/entity/Q108327439,fungus,http://www.wikidata.org/entity/Q764,,Rigidoporus liebmannii,NaN
468071,1803055080,NaN,NaN,pt,CC_BY_NC_4_0,2023-04-06T19:41:21Z,"Instituto de Botânica, São Paulo",NaN,SP-Fungi - Maria Eneyda Pacheco Kauffmann Fidalgo,Collection,...,NE,NaN,2544558,Rigidoporus liebmannii,http://www.wikidata.org/entity/Q108327439,fungus,http://www.wikidata.org/entity/Q764,,Rigidoporus liebmannii,NaN
468072,1803055051,NaN,NaN,pt,CC_BY_NC_4_0,2023-04-06T19:41:48Z,"Instituto de Botânica, São Paulo",NaN,SP-Fungi - Maria Eneyda Pacheco Kauffmann Fidalgo,Collection,...,NE,NaN,5237620,Polyporus aculeifera,http://www.wikidata.org/entity/Q108273395,fungus,http://www.wikidata.org/entity/Q764,,Polyporus aculeifera,NaN
468073,1803055032,NaN,NaN,pt,CC_BY_NC_4_0,2023-04-06T19:41:12Z,"Instituto de Botânica, São Paulo",NaN,SP-Fungi - Maria Eneyda Pacheco Kauffmann Fidalgo,Collection,...,NE,NaN,2520540,Phellinus scruposus,http://www.wikidata.org/entity/Q105060807,fungus,http://www.wikidata.org/entity/Q764,,Phellinus scruposus,NaN


In [38]:
gbifRDF = Graph()
WD = Namespace("http://www.wikidata.org/entity/")
WDT = Namespace("http://www.wikidata.org/prop/direct/")
gbifRDF.bind("wd", WD)
gbifRDF.bind("wdt", WDT)
gbifRDF.bind("dcterms", DCTERMS)
gbifRDF.bind("geo", Namespace("http://www.opengis.net/ont/geosparql#"))

for index, row in tqdm.tqdm(merged_df2.iterrows()):
    GBIFURI = URIRef("https://gbif.semscape.org/occurrence/"+str(row["gbifID"]))
    gbifRDF.add((GBIFURI, WDT.P854 , URIRef("https://www.gbif.org/occurrence/"+str(row["gbifID"]))))
    gbifRDF.add((GBIFURI, DCTERMS.identifier , Literal(str(row["gbifID"]))))

    # species
    tempkey = str(row['taxonKey']).replace('.0', '')
    if tempkey in wdtaxqid.keys():
        gbifRDF.add((GBIFURI, WDT.P225, URIRef(wdtaxqid[tempkey])))
        gbifRDF.add((URIRef(wdtaxqid[tempkey]), RDFS.label, Literal(row["taxonname"], lang="en")))

    # lat long
    try:
        if not math.isnan(float(row["decimalLatitude"])):
            gbifRDF.add((GBIFURI, WDT.P625, Literal("Point("+str(row["decimalLongitude"])+" "+str(row["decimalLatitude"])+")", datatype="http://www.opengis.net/ont/geosparql#wktLiteral")))
    except (TypeError, ValueError) as e:
        print(f"Error for gbifID={row['gbifID']}, Latitude={row['decimalLatitude']}, Longitude={row['decimalLongitude']}")
        print(f"Error details: {e}")
        print(f"Error occurred in this line: {e.__traceback__.tb_lineno}")

    # type
    typeDict = {
    "Collection": "Q2668072","Colletion": "Q2668072","collection": "Q2668072",
    "Event": "Q1656682",
    "Objeto físico": "Q223557","Physical Object": "Q223557","Physical object": "Q223557","PhysicalObject": "Q223557","PhysicalSpecimen": "Q223557","http://purl.org/dc/dcmitype/PhysicalObject": "Q223557",
    "Sound": "Q11442",
    "StillImage": "Q478798","http://purl.org/dc/dcmitype/StillImage": "Q478798",
    "Text": "Q1145976","text": "Q1145976"
}
    if row["type"] in typeDict.keys():
        gbifRDF.add((GBIFURI, WDT.P31, URIRef("http://www.wikidata.org/entity/"+typeDict[row["type"]])))
    else:
        pass
        # print(f"Error for gbifID={row['gbifID']}, type={row['type']}")

    # language
    langDict = {
        "EN": "Q1860", "en": "Q1860","En": "Q1860", "English": "Q1860",
        "PT": "Q5146","português": "Q5146",
        "es": "Q1321",
        "nl": "Q7411",
        "fr": "Q150", "FR": "Q150",
        "Latin": "Q397", "la": "Q397"
    }

    if not pd.isna(row["language"]) and row["language"] in langDict:
        gbifRDF.add((GBIFURI, WDT.P407, URIRef("http://www.wikidata.org/entity/"+langDict[row["language"]])))
    else:
        pass
        # print(f"Error for gbifID={row['gbifID']}, language={row['language']}")

    # IUCN Red List Category
    iucnDict = {
        "NE": "Q3350324", "Not Evaluated": "Q3350324",
        "LC": "Q211005", "Least Concern": "Q211005",
        "NT": "Q719675", "Near Threatened": "Q719675",
        "VU": "Q278113", "Vulnerable": "Q278113",
        "EN": "Q11394", "Endangered": "Q11394",
        "CR": "Q219127", "Critically Endangered": "Q219127",
        "EW": "Q239509", "Extinct in the Wild": "Q239509",
        "EX": "Q237350", "Extinct": "Q237350",
        "DD": "Q3245245", "Data Deficient": "Q3245245"
    }
    if not pd.notna(row["iucnRedListCategory"]) and row["iucnRedListCategory"] in iucnDict:
        gbifRDF.add((GBIFURI, WDT.P141, URIRef("http://www.wikidata.org/entity/"+iucnDict[row["iucnRedListCategory"]])))
    else:
        pass
        # print(f"Error for gbifID={row['gbifID']}, iucnRedListCategory={row['iucnRedListCategory']}")

    # taxonank
    taxonrankDict = {
        "class": "Q37517", "Class": "Q37517",
        "family": "Q35409", "Family": "Q35409",
        "genus": "Q34740", "Genus": "Q34740",
        "kingdom": "Q36732", "Kingdom": "Q36732",
        "order": "Q36602", "Order": "Q36602",
        "phylum": "Q38348", "Phylum": "Q38348",
        "species": "Q7432", "Species": "Q7432",
        "subspecies": "Q68947", "Subspecies": "Q68947",
        "variety": "Q767728", "Variety": "Q767728",
        "form": "Q279749", "Form": "Q279749"
    }

    if not pd.isna(row["taxonRank"]) and row["taxonRank"] in taxonrankDict:
        gbifRDF.add((GBIFURI, WDT.P105, URIRef("http://www.wikidata.org/entity/"+taxonrankDict[row["taxonRank"]])))
    else:
        pass
        # print(f"Error for gbifID={row['gbifID']}, taxonRank={row['taxonRank']}")
    # license
    licenseDict = {
        "CC_BY_NC_ND_2_5": "Q19113746", "http://creativecommons.org/licenses/by-nc-nd/2.5/": "Q19113746",
        "CC_BY_NC_ND_4_0": "Q24082749", "http://creativecommons.org/licenses/by-nc-nd/4.0/": "Q24082749",
        "CC_BY_NC_SA_2_5": "Q19068212", "http://creativecommons.org/licenses/by-nc-sa/2.5/": "Q19068212",
        "CC_BY_NC_SA_3_0": "Q15643954", "http://creativecommons.org/licenses/by-nc-sa/3.0/": "Q15643954",
        "CC_BY_NC_SA_4_0": "Q42553662", "http://creativecommons.org/licenses/by-nc-sa/4.0/": "Q42553662",
        "CC_BY_NC_4_0": "Q24082749", "http://creativecommons.org/licenses/by-nc/4.0/": "Q24082749",
        "CC_BY_NC_3_0": "Q18810331", "http://creativecommons.org/licenses/by-nc/3.0/": "Q18810331",
        "CC_BY_ND_2_5": "Q18810338", "http://creativecommons.org/licenses/by-nd/2.5/": "Q18810338",
        "CC_BY_ND_3_0": "Q18810160", "http://creativecommons.org/licenses/by-nd/3.0/": "Q18810160",
        "CC_BY_ND_4_0": "Q36795408", "http://creativecommons.org/licenses/by-nd/4.0/": "Q36795408",
         "CC_BY_SA_3_0": "Q14946043","http://creativecommons.org/licenses/by-sa/3.0/": "Q14946043",
        "CC_BY_SA_4_0": "Q18199165", "http://creativecommons.org/licenses/by-sa/4.0/": "Q18199165",
        "CC_BY_4_0": "Q18199165", "http://creativecommons.org/licenses/by/4.0/": "Q18199165",
        "CC_BY_3_0": "Q14947546", "http://creativecommons.org/licenses/by/3.0/": "Q14947546",
        "CC0_1_0": "Q18199165", "http://creativecommons.org/publicdomain/zero/1.0/": "Q18199165",
        "https://huh.harvard.edu/access-digital-reproductions-works-public-domain":"Q18199165"
    }
    if not pd.isna(row["license"]) and row["license"] in licenseDict:
        gbifRDF.add((GBIFURI, WDT.P275, URIRef("http://www.wikidata.org/entity/"+licenseDict[row["license"]])))
    else:
        pass
        # print(f"Error for gbifID={row['gbifID']}, license={row['license']}")

    # sex
    sexDict = {
        "FEMALE": "Q43445",
        "MALE": "Q44148"
    }
    if not pd.isna(row["sex"]) and row["sex"] in sexDict:
        gbifRDF.add((GBIFURI, WDT.P21, URIRef("http://www.wikidata.org/entity/"+sexDict[row["sex"]])))
    else:
        pass

    # recordedByID
    if not pd.isna(row["recordedByID"]):
        gbifRDF.add((GBIFURI, WDT.P170, Literal(str(row["recordedByID"]), datatype=XSD.string)))
    else:
        pass

    # associatedSequences
    if not pd.isna(row["associatedSequences"]):
        gbifRDF.add((GBIFURI, WDT.P4333, Literal(str(row["associatedSequences"]), datatype=XSD.string)))
    else:
        pass

    # occurrenceID
    if not pd.isna(row["occurrenceID"]):
        gbifRDF.add((GBIFURI, DC.identifier, Literal(str(row["occurrenceID"]), datatype=XSD.string)))

    # publisher
    if row["publisher"] in instituteQID.keys():
        gbifRDF.add((GBIFURI, WDT.P123, URIRef("http://www.wikidata.org/entity/"+instituteQID[row["publisher"]])))
        gbifRDF.add((URIRef("http://www.wikidata.org/entity/"+instituteQID[row["publisher"]]), RDFS.label, Literal(row["publisher"], lang="en")))

    # kingfom
    if not pd.isna(row["kingdom_y"]):
        gbifRDF.add((GBIFURI, WDT.P910, URIRef(row["kingdom_y"]))) #chapter
        gbifRDF.add((URIRef(row["kingdom_y"]), RDFS.label, Literal(row["kingdomname"], lang="en")))
    else:
        pass

    #Wiki article
    if not pd.isna(row["article"]):
        gbifRDF.add((GBIFURI, DC.description, URIRef(row["article"]))) #chapter
    else:
        pass

468075it [02:37, 2970.30it/s]


In [39]:
df_image = pd.read_csv('multimedia.csv', sep=';', on_bad_lines = 'warn')
for index, row in df_image.iterrows():
    GBIFURI = URIRef("https://gbif.semscape.org/occurrence/"+str(row["gbifID"]))
    mediaURI = BNode()
    if row["type"] == "StillImage":
        gbifRDF.add((mediaURI, WDT.P31, WD.Q17538176))
    elif row["type"] == "Sound":
        gbifRDF.add((mediaURI, WDT.P31, WD.Q11461))
    gbifRDF.add((mediaURI, DCTERMS.format, Literal(row["format"], datatype=XSD.string)))
    if not pd.isna(row["license"]) and row["license"] in licenseDict:
        gbifRDF.add((mediaURI, WDT.P275, URIRef("http://www.wikidata.org/entity/"+licenseDict[row["license"]])))
    else:
        pass
    gbifRDF.add((mediaURI, WDT.P2699, Literal(row["identifier"], datatype=XSD.string)))
    gbifRDF.add((mediaURI, WDT.P361, GBIFURI))

In [40]:
wdtaxqid

{'3152764': 'http://www.wikidata.org/entity/Q15375663',
 '3993719': 'http://www.wikidata.org/entity/Q15544698',
 '3023040': 'http://www.wikidata.org/entity/Q15545138',
 '2703717': 'http://www.wikidata.org/entity/Q161800',
 '2758930': 'http://www.wikidata.org/entity/Q1503476',
 '2304539': 'http://www.wikidata.org/entity/Q3167563',
 '2286258': 'http://www.wikidata.org/entity/Q3792820',
 '2355086': 'http://www.wikidata.org/entity/Q5396646',
 '7264791': 'http://www.wikidata.org/entity/Q6067513',
 '3698590': 'http://www.wikidata.org/entity/Q12199959',
 '3157522': 'http://www.wikidata.org/entity/Q15356845',
 '3860304': 'http://www.wikidata.org/entity/Q15390296',
 '3617905': 'http://www.wikidata.org/entity/Q15393995',
 '5549398': 'http://www.wikidata.org/entity/Q15482967',
 '2650880': 'http://www.wikidata.org/entity/Q15601786',
 '2738902': 'http://www.wikidata.org/entity/Q221543',
 '2738136': 'http://www.wikidata.org/entity/Q3292034',
 '1264155': 'http://www.wikidata.org/entity/Q2028579',
 '5

In [10]:
# Define the URI prefix you are interested in
prefix = "http://www.wikidata.org/entity/"

# Initialize a set to hold all matching URIs
wikidata_uris = set()

# Iterate through all nodes in the graph, both subjects and objects
for node in gbifRDF.all_nodes():
    if isinstance(node, URIRef) and str(node).startswith(prefix):
        wikidata_uris.add(str(node))


NameError: name 'gbifRDF' is not defined

In [54]:
gbifRDF.serialize(destination='gbifSuriname.ttl', format='turtle', encoding="utf-8")

<Graph identifier=N015c5699a1834fccab9016545983a6d6 (<class 'rdflib.graph.Graph'>)>

In [12]:
wdGraph = Graph()

# Wrapping wikidata_uris with tqdm to create a progress bar
for wduri in tqdm(wikidata_uris, desc="Processing URIs"):
    try:
        print(str(wduri))
        wdGraph.parse(str(wduri), format='turtle')
    except Exception as e:  # It's a good practice to log the exception
        print(f"An error occurred while parsing {wduri}: {e}")
        continue

NameError: name 'Graph' is not defined

In [44]:
taxonpages = dict()
query = """
PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    SELECT DISTINCT  ?taxon ?taxonLabel ?publisher ?publisherLabel ?observation ?gbifObservation ?media ?media_url ?license WHERE {
           ?media wdt:P2699 ?media_url ;
                  wdt:P275 ?license ;
                  wdt:P361 ?observation .
          ?observation wdt:P225 ?taxon ;
                       wdt:P854 ?gbifObservation ;
                       wdt:P123 ?publisher .
          ?taxon rdfs:label ?taxonLabel .
          OPTIONAL {?taxon dc:description ?article .}
          ?publisher rdfs:label ?publisherLabel .
            }"""
# Assuming gbifRDF.query(query) has been defined and executed
results = gbifRDF.query(query)
taxonpages = {}  # Ensure that taxonpages is defined before the loop

for row in results:
    # Cache commonly used row items to avoid repetitive dictionary lookups and conversions
    taxon_label = str(row["taxonLabel"])
    publisher_label = str(row["publisherLabel"])

    # If the taxon label is not in taxonpages, initialize it with necessary keys
    if taxon_label not in taxonpages:
        taxonpages[taxon_label] = {
            "wd_uri": str(row["taxon"]),
            "publisher": {},  # Initialize the publisher dictionary here
        }
        # Add the article if it exists in the row
        if "article" in row:
            taxonpages[taxon_label]["article"] = str(row["article"])

    # Ensure the publisher label is initialized in the taxon's publisher dictionary
    if publisher_label not in taxonpages[taxon_label]["publisher"]:
        taxonpages[taxon_label]["publisher"][publisher_label] = {
            "observations": []  # Initialize the observations list here
        }

    # Construct the observation dictionary
    observation = {
        "observation_id": str(row["gbifObservation"]),
        "media": [str(row["media_url"])],  # Initialize media as a list with the media_url
        "license": str(row["license"])
    }
    # Append the observation dictionary to the observations list
    taxonpages[taxon_label]["publisher"][publisher_label]["observations"].append(observation)

# The rest of your processing can continue after this




In [ ]:
taxonpages

In [57]:
import shutil
import os
shutil.rmtree('./taxa/')
os.makedirs(os.path.dirname('./taxa/'), exist_ok=True)


In [58]:
for taxon in taxonpages.keys():
    mdFile = MdUtils(file_name='taxa/'+ taxon.replace(" ", "_"), title=taxon+" ("+taxonpages[taxon]["wd_uri"].replace("http://www.wikidata.org/entity/", "")+")")

    if URIRef(row["taxon"]) in taxoninformation.keys():
        if "wikiintroduction" in taxoninformation[URIRef(row["taxon"])].keys():
            mdFile.new_line(taxoninformation[URIRef(row["taxon"])]["wikiintroduction"])
            mdFile.new_line(f"Read more on [English Wikipedia](https://en.wikipedia.org/page/{row['taxonLabel'].replace('' , '_')})")
    mdFile.new_line(f"Visualize Wikidata on [Scholia](https://scholia.toolforge.org/taxon/{taxonpages[taxon]['wd_uri'].replace('http://www.wikidata.org/entity/', '')})")
    for organisation in taxonpages[taxon]["publisher"].keys():
        mdFile.new_header(level=1, title=organisation)
        for observation in taxonpages[taxon]["publisher"][organisation]["observations"]:
            mdFile.new_line(f"Observation: [{observation['observation_id']}]({observation['observation_id']})")
            mdFile.new_line(f"License: [{observation['license']}]({observation['license']})")
            for media in observation["media"]:
                if media != "nan":
                    mdFile.new_line(f"![{observation['observation_id']}]({media.replace('square', 'medium')})")
    mdFile.create_md_file()


In [51]:
# Assuming `result_df` is your DataFrame and it has columns named 'article' and 'taxonname'
taxonname_list = result_df[result_df["article"] != ""]["taxonname"].tolist()
taxonname_list

['Bromus secalinus',
 'Aframomum melegueta',
 'Cerodrillia thea',
 'Caryota mitis',
 'Astrocaryum jauari',
 'Colibri delphinae',
 'Rhizoprionodon terraenovae',
 'Ostichthys trachypoma',
 'Lafoensia pacari',
 'Nucula atacellana',
 'Mitra straminea',
 'Asplenium serratum',
 'Funiculina quadrangularis',
 'Vitex orinocensis',
 'Astrocaryum vulgare',
 'Diospyros artanthifolia',
 'Geophagus brasiliensis',
 'Philydor erythrocercum',
 'Phoenicoprocta vacillans',
 'Elaenia ruficeps',
 'Leptodactylus mystaceus',
 'Pouteria glomerata',
 'Lobatus gigas',
 'Bubo virginianus',
 'Anopheles albimanus',
 'Herpsilochmus stictocephalus',
 'Laportea aestuans',
 'Carex distans',
 'Coleomegilla maculata',
 'Suiriri suiriri',
 'Calamus penna',
 'Strombus alatus',
 'Dismorphia amphione',
 'Leucosyrinx subgrundifera',
 'Bradypus tridactylus',
 'Eumops trumbulli',
 'Tinamus major',
 'Exocoetus obtusirostris',
 'Allium oreoprasum',
 'Phyllonotus globosus',
 'Ixora panurensis',
 'Lepidochelys olivacea',
 'Copelat

In [56]:
import yaml

with open(r'_toc.yml') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    species = yaml.load(file, Loader=yaml.FullLoader)

species["parts"][0]["chapters"] = []
for file in os.listdir('./taxa/'):
    species["parts"][0]["chapters"].append({'file': 'taxa/'+file})
with open(r'_toc.yml', 'w+') as file:
    documents = yaml.dump(species, file)

## Below is for reference only

In [6]:
# Assuming `notFound` is your dictionary
data = []  # List to hold data for DataFrame

for institute, details in notFound.items():
    try:
        # Access the first result for the current institute
        if len(details["results"]) == 0:
            # If no results are found, append a dictionary with the institute and an error message
            print(f"No results found for {institute}")
            data.append({
                "Institute": "FOUT" + institute,
                "Error": "No results found"
            })
            continue
        first_result = details["results"][0]

        # Append a dictionary with the required information to the data list
        data.append({
            "Institute": institute,
            "Key": first_result.get("key", None),
            "Title": first_result.get("title", None),
            "Homepage": first_result.get("homepage", None),
            "City": first_result.get("city", None),
            "Country": first_result.get("country", None),
            "PostalCode": first_result.get("postalCode", None),
            "Address": first_result.get("address", None),
            "Phone": first_result.get("phone", None),
            "Email": first_result.get("email", None),
            "Endpoints": first_result.get("endpoints", None)
        })
    except KeyError as e:
        # If some data is not found, append a dictionary with the institute and an error message
        print(f"Error retrieving data for {institute}: {e}")
        data.append({
            "Institute": "FOUT" + institute,
            "Error": details["results"]
        })

# Create DataFrame from the data list
dfinstitutestoBuild = pd.DataFrame(data)
dfinstitutestoBuild


No results found for CBGP (UMR INRA, Cirad, IRD, Montpellier SupAgro)


,Institute,Key,Title,Homepage,City,Country,PostalCode,Address,Phone,Email,Endpoints,Error
0,"Walter Reed Biosystematics Unit, Smithsonian I...",5ab24120-de25-11dc-9a41-b8a03c50a862,"Walter Reed Biosystematics Unit, Smithsonian I...","[http://worldviewsolutions.net:8080, https://w...",Suitland,US,20746-2863,[4210 Silver Hill Road],[],[nmnh-wrbu@si.edu],[],NaN
1,"FOUTCBGP (UMR INRA, Cirad, IRD, Montpellier Su...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No results found


In [7]:
df.groupby('license').count()

,gbifID,accessRights,bibliographicCitation,language,modified,publisher,references,rightsHolder,type,institutionID,...,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name,iucnRedListCategory,eventType
license,,,,,,,,,,,,,,,,,,,,,
CC0_1_0,186419,18231,6154,23059,122614,186419,17626,144754,24170,173248,...,57948,57948,57946,57946,57945,57945,24,24,159478,0
CC_BY_4_0,228538,26003,27864,8228,44970,228538,13366,38612,39455,43029,...,184420,184420,184420,184420,184420,184420,2,2,204026,0
CC_BY_NC_4_0,143578,5800,8975,8812,97185,143578,77941,87255,18263,24001,...,103846,103846,103846,103846,103846,103846,0,0,123378,0


In [8]:
df.groupby('iucnRedListCategory').count()

,gbifID,accessRights,bibliographicCitation,language,license,modified,publisher,references,rightsHolder,type,...,projectId,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name,eventType
iucnRedListCategory,,,,,,,,,,,,,,,,,,,,,
CR,694,93,17,113,694,575,694,118,508,137,...,16,106,106,106,106,106,106,0,0,0
DD,2437,63,43,184,2437,1390,2437,232,1399,238,...,190,803,803,803,803,803,803,1,1,0
EN,1856,161,903,177,1856,1428,1856,114,701,1103,...,29,250,250,250,250,250,250,0,0,0
LC,316239,12303,15716,14942,316239,136023,316239,78108,141498,32182,...,9567,235827,235827,235827,235827,235826,235826,7,7,0
NE,158507,17228,20618,14571,158507,75733,158507,19373,91789,37729,...,7712,71845,71845,71843,71843,71843,71843,14,14,0
NT,4707,70,577,110,4707,1998,4707,1144,1530,680,...,479,3231,3231,3231,3231,3231,3231,1,1,0
VU,2442,131,272,225,2442,1253,2442,372,1247,524,...,192,1077,1077,1077,1077,1077,1077,0,0,0


In [9]:
df.groupby('higherClassification').count()

,gbifID,accessRights,bibliographicCitation,language,license,modified,publisher,references,rightsHolder,type,...,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name,iucnRedListCategory,eventType
higherClassification,,,,,,,,,,,,,,,,,,,,,
Sphyrnidae,2,0,0,0,2,2,2,0,2,0,...,0,0,0,0,0,0,0,0,2,0
(Hedw.) Mitt.,1,0,0,0,1,1,1,0,0,1,...,1,1,1,1,1,1,0,0,0,0
(Hook. & Kunth) Spruce ex J.Florsch.,1,0,0,0,1,1,1,0,0,1,...,1,1,1,1,1,1,0,0,0,0
(Hook.) H.A.Crum & Steere,1,0,0,0,1,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
(Hornsch.) Wijk & Margad.,1,0,0,0,1,1,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
| Megaceryle,2,2,2,2,2,2,2,2,0,2,...,0,0,0,0,0,0,0,0,2,0
| Phaethornis,1,1,1,1,1,1,1,1,0,1,...,0,0,0,0,0,0,0,0,1,0
| Porphyrio,5,5,5,5,5,5,5,5,0,5,...,0,0,0,0,0,0,0,0,5,0


In [10]:
df.groupby('species').count()[["higherClassification"]]

,higherClassification
species,
Abaeis albula,0
Abathomphalus mayaroensis,0
Abelmoschus esculentus,41
Abelmoschus moschatus,10
Ablennes hians,1
...,...
Zygopa michaelis,1
Zygosepalum labiosum,2
Zygostates ligulata,1


In [11]:
df.groupby('species').count()

,gbifID,accessRights,bibliographicCitation,language,license,modified,publisher,references,rightsHolder,type,...,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name,iucnRedListCategory,eventType
species,,,,,,,,,,,,,,,,,,,,,
Abaeis albula,1,0,0,0,1,1,1,1,1,0,...,1,1,1,1,1,1,0,0,1,0
Abathomphalus mayaroensis,27,0,0,0,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,27,0
Abelmoschus esculentus,42,0,0,0,42,4,42,1,40,2,...,35,35,35,35,35,35,0,0,42,0
Abelmoschus moschatus,12,0,2,0,12,1,12,0,11,2,...,5,5,5,5,5,5,0,0,12,0
Ablennes hians,3,0,0,1,3,3,3,1,0,1,...,0,0,0,0,0,0,0,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zygopa michaelis,5,0,0,0,5,1,5,0,1,0,...,0,0,0,0,0,0,0,0,5,0
Zygosepalum labiosum,4,0,0,0,4,1,4,0,1,0,...,1,1,1,1,1,1,0,0,4,0
Zygostates ligulata,1,0,0,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [12]:
df.groupby('sex').count()

,gbifID,accessRights,bibliographicCitation,language,license,modified,publisher,references,rightsHolder,type,...,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name,iucnRedListCategory,eventType
sex,,,,,,,,,,,,,,,,,,,,,
FEMALE,23174,4452,1651,5984,23174,20788,23174,2587,18336,4926,...,7929,7929,7929,7929,7929,7929,2,2,15642,0
MALE,32686,4992,1906,6620,32686,29629,32686,3610,26576,5511,...,9311,9311,9311,9311,9311,9311,2,2,23665,0


In [13]:
df.groupby('taxonRank').count()

,gbifID,accessRights,bibliographicCitation,language,license,modified,publisher,references,rightsHolder,type,...,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name,iucnRedListCategory,eventType
taxonRank,,,,,,,,,,,,,,,,,,,,,
CLASS,1419,20,21,174,1419,278,1419,29,263,193,...,195,195,195,195,195,195,0,0,1056,0
FAMILY,16230,6164,1891,1241,16230,9040,16230,1067,3306,2310,...,9308,9308,9308,9308,9308,9308,0,0,10053,0
FORM,136,6,17,15,136,46,136,17,81,47,...,30,30,30,30,30,30,0,0,0,0
GENUS,48111,17388,4378,5014,48111,29982,48111,2547,15705,8495,...,25426,25426,25426,25426,25426,25426,0,0,16288,0
KINGDOM,2680,45,145,81,2680,1302,2680,116,1184,236,...,590,590,590,590,590,590,0,0,597,0
ORDER,1056,260,42,85,1056,407,1056,29,116,84,...,375,375,375,375,375,375,0,0,863,0
PHYLUM,1470,44,121,11,1470,281,1470,44,153,129,...,84,84,84,84,84,84,0,0,746,0
SPECIES,457137,22122,33324,27203,457137,202637,457137,97413,227321,65371,...,296559,296559,296557,296557,296556,296556,23,23,456656,0
SUBSPECIES,23600,3794,2259,2984,23600,19607,23600,7157,19467,3461,...,9369,9369,9369,9369,9369,9369,3,3,538,0


In [14]:
df.groupby('publishingCountry').count()

,gbifID,accessRights,bibliographicCitation,language,license,modified,publisher,references,rightsHolder,type,...,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name,iucnRedListCategory,eventType
publishingCountry,,,,,,,,,,,,,,,,,,,,,
6,1,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AR,86,0,0,31,86,75,86,10,10,31,...,45,45,45,45,45,45,0,0,84,0
AT,622,0,0,0,622,0,622,0,0,0,...,64,64,64,64,64,64,0,0,562,0
AU,447,2,1,37,447,284,447,1,230,281,...,32,32,32,32,32,32,0,0,266,0
BE,7977,0,48,3085,7977,7478,7977,4887,7926,7922,...,871,871,871,871,871,871,0,0,5646,0
BG,12,0,0,0,12,0,12,10,0,0,...,7,7,7,7,7,7,0,0,12,0
BR,9463,1143,4339,1566,9463,3925,9463,0,9424,8265,...,2494,2494,2494,2494,2494,2494,0,0,8954,0
CA,4898,4587,347,4830,4898,4716,4898,35,4896,4848,...,3590,3590,3590,3590,3590,3590,0,0,4634,0
CH,1660,0,0,0,1660,57,1660,1117,57,57,...,652,652,652,652,652,652,0,0,1532,0


In [20]:
result_df.groupby("kingdomname").count()

,taxonId,taxonname,taxon,kingdom,article
kingdomname,,,,,
Chromista,116,116,116,116,116
Monera,3,3,3,3,3
animal,8562,8562,8562,8562,8562
bacteria,3,3,3,3,3
fungus,612,612,612,612,612
plant,8209,8209,8209,8209,8209
protist,12,12,12,12,12
protozoa,12,12,12,12,12


In [26]:
merged_df[merged_df['taxonId']==""][['taxonKey', 'taxonId', 'taxonname', 'taxon', 'genericName', 'specificEpithet']].drop_duplicates()

,taxonKey,taxonId,taxonname,taxon,genericName,specificEpithet
5142,7499281.0,,Casearia pitumba,http://www.wikidata.org/entity/Q15389271,Casearia,pitumba
5428,2978125.0,,Copaifera guianensis,http://www.wikidata.org/entity/Q107054779,Copaifera,guianensis
8647,2925401.0,,Avicennia nitida,http://www.wikidata.org/entity/Q87612389,Avicennia,nitida
8656,7715585.0,,Avicennia nitida,http://www.wikidata.org/entity/Q87612389,Avicennia,nitida
8969,5248141.0,,Polyporus fomentarius,http://www.wikidata.org/entity/Q105065407,Polyporus,fomentarius
...,...,...,...,...,...,...
466793,7285715.0,,Heteropterys suberosa,http://www.wikidata.org/entity/Q111094489,Heteropterys,suberosa
466823,8375315.0,,Auplopus stagei,http://www.wikidata.org/entity/Q117823629,Auplopus,stagei
466842,2717425.0,,Eleocharis ochreata,http://www.wikidata.org/entity/Q110437399,Eleocharis,ochreata
466865,5565418.0,,Myrica javanica,http://www.wikidata.org/entity/Q50844166,Myrica,javanica


In [16]:
    wdtaxqid

{'2241784': 'http://www.wikidata.org/entity/Q3016830',
 '3992233': 'http://www.wikidata.org/entity/Q15334447',
 '2514762': 'http://www.wikidata.org/entity/Q2117089',
 '2485721': 'http://www.wikidata.org/entity/Q1270918',
 '2286513': 'http://www.wikidata.org/entity/Q105972189',
 '2929028': 'http://www.wikidata.org/entity/Q15539093',
 '3086328': 'http://www.wikidata.org/entity/Q15606320',
 '7304240': 'http://www.wikidata.org/entity/Q15609790',
 '5415073': 'http://www.wikidata.org/entity/Q17682523',
 '5253985': 'http://www.wikidata.org/entity/Q10600153',
 '2221803': 'http://www.wikidata.org/entity/Q4470463',
 '2352597': 'http://www.wikidata.org/entity/Q6405418',
 '6004674': 'http://www.wikidata.org/entity/Q14929871',
 '2900288': 'http://www.wikidata.org/entity/Q7396079',
 '2681226': 'http://www.wikidata.org/entity/Q17199991',
 '3823448': 'http://www.wikidata.org/entity/Q110583725',
 '1837540': 'http://www.wikidata.org/entity/Q13632704',
 '3055755': 'http://www.wikidata.org/entity/Q1537070

In [29]:
df_image.groupby("license").count()

,gbifID,type,format,identifier,references,title,description,source,audience,created,creator,contributor,publisher,rightsHolder
license,,,,,,,,,,,,,,
(c) Field Museum of Natural History,24,24,24,24,0,24,24,0,0,23,24,0,0,24
(c) Field Museum of Natural History - CC BY-NC 4.0,3396,3396,3396,3396,0,3396,3396,0,0,3124,3366,0,662,3396
Attribution-ShareAlike (BY-SA) Creative Commons License and GNU Free Documentation License (GFDL),408,408,408,408,408,408,0,0,0,0,408,0,0,408
"Copyright Borror Lab of Bioacoustics, Ohio State University 2018",1,1,1,1,0,1,1,0,0,0,0,0,0,1
Copyright permission not set,10,10,10,10,0,0,4,0,0,0,10,0,10,0
Creative Commons,13,0,0,0,13,0,0,0,0,0,0,0,0,13
Daniel Barthelemy (cc-by-nc),120,120,120,120,0,120,120,120,0,120,120,0,0,120
"Partial images provided by this server are released under the Creative Commons cc-by-sa 3.0 (generic) licence [https://creativecommons.org/licenses/by-sa/3.0/de/]. Please credit images to BGBM following our citation guidelines [https://ww2.bgbm.org/Herbarium/cite.cfm]. If you would like to use images in a format or resolution which is not provided here, please contact us (d.roepert[at]bgbm.org).",313,165,313,313,313,0,0,0,0,0,0,0,0,0
Usage Conditions Apply,2952,2952,2952,2952,0,2952,2208,2223,0,0,2843,0,2952,0


In [30]:
df_image.groupby("type").count()

,gbifID,format,identifier,references,title,description,source,audience,created,creator,contributor,publisher,license,rightsHolder
type,,,,,,,,,,,,,,
InteractiveResource,4487,4487,4487,0,4461,4487,0,0,0,4487,0,0,4487,4487
Sound,1314,1314,1314,6,1,1303,0,0,6,1313,0,6,1314,1314
StillImage,138453,138446,138079,18099,33464,83748,13513,0,15358,63244,0,26913,130308,111377


In [31]:
df_image[df_image["type"]=="Sound"]

,gbifID,type,format,identifier,references,title,description,source,audience,created,creator,contributor,publisher,license,rightsHolder
8407,2243677327,Sound,audio/mpeg,https://xeno-canto.org/sounds/uploaded/OOECIWC...,NaN,NaN,61 s,NaN,NaN,NaN,Peter Boesman,NaN,NaN,http://creativecommons.org/licenses/by-nc-nd/4.0/,Peter Boesman
8412,2243677692,Sound,audio/mpeg,https://xeno-canto.org/sounds/uploaded/OOECIWC...,NaN,NaN,142 s,NaN,NaN,NaN,Peter Boesman,NaN,NaN,http://creativecommons.org/licenses/by-nc-nd/4.0/,Peter Boesman
8415,2243677773,Sound,audio/mpeg,https://xeno-canto.org/sounds/uploaded/OOECIWC...,NaN,NaN,43 s,NaN,NaN,NaN,Peter Boesman,NaN,NaN,http://creativecommons.org/licenses/by-nc-nd/4.0/,Peter Boesman
8416,2243677804,Sound,audio/mpeg,https://xeno-canto.org/sounds/uploaded/OOECIWC...,NaN,NaN,52 s,NaN,NaN,NaN,Peter Boesman,NaN,NaN,http://creativecommons.org/licenses/by-nc-nd/4.0/,Peter Boesman
8419,2243677831,Sound,audio/mpeg,https://xeno-canto.org/sounds/uploaded/OOECIWC...,NaN,NaN,25 s,NaN,NaN,NaN,Peter Boesman,NaN,NaN,http://creativecommons.org/licenses/by-nc-nd/4.0/,Peter Boesman
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151524,4173214802,Sound,audio/vnd.wave,https://xeno-canto.org/sounds/uploaded/OTVUCEG...,NaN,NaN,19 s,NaN,NaN,NaN,Hans Groot,NaN,NaN,http://creativecommons.org/licenses/by-nc-sa/4.0/,Hans Groot
151525,4173215545,Sound,audio/mpeg,https://xeno-canto.org/sounds/uploaded/OTVUCEG...,NaN,NaN,4 s,NaN,NaN,NaN,Hans Groot,NaN,NaN,http://creativecommons.org/licenses/by-nc-sa/4.0/,Hans Groot
151530,4173216823,Sound,audio/vnd.wave,https://xeno-canto.org/sounds/uploaded/OTVUCEG...,NaN,NaN,12 s,NaN,NaN,NaN,Hans Groot,NaN,NaN,http://creativecommons.org/licenses/by-nc-sa/4.0/,Hans Groot
151532,4173217552,Sound,audio/vnd.wave,https://xeno-canto.org/sounds/uploaded/OTVUCEG...,NaN,NaN,7 s,NaN,NaN,NaN,Hans Groot,NaN,NaN,http://creativecommons.org/licenses/by-nc-sa/4.0/,Hans Groot


In [22]:
df_image.groupby('identifier').count()

,gbifID,type,format,references,title,description,source,audience,created,creator,contributor,publisher,license,rightsHolder
identifier,,,,,,,,,,,,,,
http://161.111.171.57/fcgi-bin/iipsrv.fcgi?FIF=/Catalogo/Fanerogamia/IMGS_web/IMGS_jp2/MA-01-00250008.jp2&WID=2200&QLT=40&CVT=jpeg,1,1,1,1,1,0,0,0,0,0,0,0,0,0
http://161.111.171.57/fcgi-bin/iipsrv.fcgi?FIF=/Catalogo/Fanerogamia/IMGS_web/IMGS_jp2/MA-01-00250662.jp2&WID=2200&QLT=40&CVT=jpeg,1,1,1,1,1,0,0,0,0,0,0,0,0,0
http://161.111.171.57/fcgi-bin/iipsrv.fcgi?FIF=/Catalogo/Fanerogamia/IMGS_web/IMGS_jp2/MA-01-00251190.jp2&WID=2200&QLT=40&CVT=jpeg,1,1,1,1,1,0,0,0,0,0,0,0,0,0
http://161.111.171.57/fcgi-bin/iipsrv.fcgi?FIF=/Catalogo/Fanerogamia/IMGS_web/IMGS_jp2/MA-01-00400057.jp2&WID=2200&QLT=40&CVT=jpeg,1,1,1,1,1,0,0,0,0,0,0,0,0,0
http://161.111.171.57/fcgi-bin/iipsrv.fcgi?FIF=/Catalogo/Fanerogamia/IMGS_web/IMGS_jp2/MA-01-00539887.jp2&WID=2200&QLT=40&CVT=jpeg,1,1,1,1,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://zenodo.org/record/2702457/files/CAM021233_v.JPG,1,1,1,0,0,0,0,0,0,0,0,0,0,0
https://zenodo.org/record/2702457/files/CAM021234_d.JPG,1,1,1,0,0,0,0,0,0,0,0,0,0,0
https://zenodo.org/record/2702457/files/CAM021234_v.JPG,1,1,1,0,0,0,0,0,0,0,0,0,0,0


In [71]:
len(gbifRDF.all_nodes())

0

In [ ]:
len (gbifRDF.all_nodes())

In [33]:
query = """
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    SELECT DISTINCT ?taxon ?taxonLabel WHERE {
           ?observation wdt:P225 ?taxon .
           ?taxon ?p ?taxonLabel .
            }"""
results = gbifRDF.query(query)
for row in results:
    print(row["taxon"], row["taxonLabel"])
    mdFile = mdutils.MdUtils(file_name='taxa/'+ row["taxonLabel"].replace(" ", "_"), title=row["taxonLabel"]+" ("+row["taxon"]+")")
    if URIRef(row["taxon"]) in taxoninformation.keys():
        if "wikiintroduction" in taxoninformation[URIRef(row["taxon"])].keys():
            mdFile.new_line(taxoninformation[URIRef(row["taxon"])]["wikiintroduction"])
            mdFile.new_line(f"Read more on [English Wikipedia](https://en.wikipedia.org/page/{row['taxonLabel'].replace('' , '_')})")
        mdFile.new_line(f"Visualize Wikidata on [Scholia](https://scholia.toolforge.org/taxon/{row['taxon'].replace('http://www.wikidata.org/entity/', '')})")

http://www.wikidata.org/entity/Q15381185 Sauvagesia erecta


NameError: name 'mdutils' is not defined

In [34]:
import mdutils
for taxon in table.keys():
    if table[taxon]["taxon_name"] in verified:
        mdFile = mdutils.MdUtils(file_name='new_articles/'+table[taxon]["taxon_name"].replace(" ", "_"),title=table[taxon]["taxon_name"])
        mdFile.new_line(f"[iNaturalist taxon id: {taxon}](https://www.inaturalist.org/taxa/{taxon})")
        for image in table[taxon]["photos"]:
            mdFile.new_line(f"![{table[taxon]['taxon_name']}]({image.replace('square', 'medium')})")
        mdFile.create_md_file()

NameError: name 'table' is not defined